# Import

In [2]:
import os
import json

import numpy as np
import tiktoken
import faiss
from openai import OpenAI
from dotenv import load_dotenv

# Initialize Client

In [3]:
# Setting client
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

DEFAULT_MODEL = os.getenv("DEFAULT_MODEL", "gpt-4o-mini")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL", "text-embedding-3-small")
MAX_COMPLETION_TOKENS = int(os.getenv("MAX_COMPLETION_TOKENS", "256"))

# Utils

In [4]:
def count_tokens(text: str, model: str = DEFAULT_MODEL):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

# Produce embeddings

## Simplest text embedding

In [6]:
text = "OpenAI 的 API 可以用來構建智慧應用，例如自動摘要與文件搜尋。"
count_tokens(text, EMBEDDING_MODEL)

37

In [7]:
response = client.embeddings.create(
    model=EMBEDDING_MODEL,
    input=text
)

embedding_vector = response.data[0].embedding
print("向量維度：", len(embedding_vector))
print("前五個值：", embedding_vector[:5])

向量維度： 1536
前五個值： [0.009911724366247654, -0.003875293303281069, 0.012393995188176632, -0.01702004484832287, 0.029492152854800224]


## Batch embeddings (using list)

In [8]:
texts = [
    "草莓是一種水果。",
    "奇異果也是水果。",
    "汽車是一種交通工具。"
]

response = client.embeddings.create(
    model=EMBEDDING_MODEL,
    input=texts
)

for i, d in enumerate(response.data):
    print(f"第 {i+1} 筆: 向量維度：{len(d.embedding)}")

第 1 筆: 向量維度：1536
第 2 筆: 向量維度：1536
第 3 筆: 向量維度：1536


In [14]:
vectors = np.array([d.embedding for d in response.data])
print(vectors.shape)

(3, 1536)


## Structure of embedding subject

In [12]:
print(json.dumps(response.data[0].model_dump(), indent=2))

{
  "embedding": [
    -0.011430488899350166,
    0.015797004103660583,
    -0.01611732691526413,
    0.027598394080996513,
    0.06274967640638351,
    -0.0025288693141192198,
    -0.025828184559941292,
    0.029672065749764442,
    0.019876912236213684,
    -0.0292674470692873,
    0.011733953841030598,
    0.006178870797157288,
    -0.017381761223077774,
    0.000446766905952245,
    -0.007852138951420784,
    -0.043462835252285004,
    -0.04336167871952057,
    -0.02222033217549324,
    -0.016016172245144844,
    0.0006785799050703645,
    0.017600929364562035,
    0.049329809844493866,
    0.046666067093610764,
    -0.012568480335175991,
    0.020534418523311615,
    -0.0002574968384578824,
    0.024698622524738312,
    0.005761607084423304,
    -0.010073329322040081,
    -0.008366342633962631,
    -0.018865365535020828,
    -0.012467325665056705,
    0.020348967984318733,
    -0.01795497164130211,
    0.0265194084495306,
    -0.01610889658331871,
    0.020736727863550186,
    -0.

# Vector Search: Using Cosine Similarity

## Use `NumPy` to calculate similarity

In [17]:
texts = [
    "台灣是水果王國。",
    "日韓朋友特別喜歡台灣的水果，尤其是芒果和香蕉。",
    "從今天開始因為颱風的影響，台北可能會下一個禮拜的雨。"
]

response = client.embeddings.create(
    model=EMBEDDING_MODEL,
    input=texts
)

vectors = np.array([d.embedding for d in response.data])

In [16]:
# Define similarity function
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [18]:
# See similarity between sentences
similarity_1_2 = cosine_similarity(vectors[0], vectors[1])
similarity_1_3 = cosine_similarity(vectors[0], vectors[2])
similarity_2_3 = cosine_similarity(vectors[1], vectors[2])

print(f"句子 1 vs 句子 2 相似度：{similarity_1_2:.3f}")
print(f"句子 1 vs 句子 3 相似度：{similarity_1_3:.3f}")
print(f"句子 2 vs 句子 3 相似度：{similarity_2_3:.3f}")

句子 1 vs 句子 2 相似度：0.564
句子 1 vs 句子 3 相似度：0.278
句子 2 vs 句子 3 相似度：0.182


## Use query to search the sentense

In [21]:
query = "哪些句子提到了芒果？"

query_vec = client.embeddings.create(
    model=EMBEDDING_MODEL,
    input=query
).data[0].embedding

In [22]:
scores = [cosine_similarity(query_vec, v) for v in vectors]
for t, s in sorted(zip(texts, scores), key=lambda x: x[1], reverse=True):
    print(f"{s:.3f} → {t}")

0.435 → 日韓朋友特別喜歡台灣的水果，尤其是芒果和香蕉。
0.352 → 台灣是水果王國。
0.133 → 從今天開始因為颱風的影響，台北可能會下一個禮拜的雨。


# Use `FAISS` to perform vector search

## Sentences

In [5]:
texts = [
    "藍靛果屬於水果類，富有豐富的維生素A、C、E及礦物質鉀、鈣、磷、鐵，是一個很棒的抗氧化來源。",
    "台灣芒果的盛產期為每年的5月至7月，主要產區集中在台南、屏東和高雄。",
    "影集《小學風雲》自2021年首播以來，便以幽默風趣卻不失真誠的敘事風格，在美劇中脫穎而出，成為第80屆金球獎大贏家。",
    "Hulu登陸台灣Disney+，正式取代原Star品牌，目的是擴展Hulu的全球影響力。",
]

## Produce vectors

In [6]:
response = client.embeddings.create(
    model=EMBEDDING_MODEL,
    input=texts
)

embeddings = np.array([d.embedding for d in response.data]).astype("float32")
print("向量形狀：", embeddings.shape)

向量形狀： (4, 1536)


## Make FAISS index (IndexFlatIP)

In [7]:
# 向量維度（text-embedding-3-small 為 1536）
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)

# 先對每個向量進行 L2 normalize（讓內積 = 餘弦相似度）
faiss.normalize_L2(embeddings)
index.add(embeddings)

print(f"已加入 {index.ntotal} 筆向量。")

已加入 4 筆向量。


## Make a search

In [10]:
query = "我想找跟小學有關的句子。"

# 將 query 轉成 embedding
query_vec = client.embeddings.create(
    model=EMBEDDING_MODEL,
    input=query
).data[0].embedding

# 轉為 numpy 並 normalize
query_vec = np.array([query_vec]).astype("float32")
faiss.normalize_L2(query_vec)

# 搜尋最相似的前 2 筆
top_k = 2
scores, indices = index.search(query_vec, top_k)

for i, idx in enumerate(indices[0]):
    print(f"Top {i+1} | 相似度：{scores[0][i]:.3f}")
    print("原文：", texts[idx])
    print()

Top 1 | 相似度：0.311
原文： 影集《小學風雲》自2021年首播以來，便以幽默風趣卻不失真誠的敘事風格，在美劇中脫穎而出，成為第80屆金球獎大贏家。

Top 2 | 相似度：0.205
原文： 藍靛果屬於水果類，富有豐富的維生素A、C、E及礦物質鉀、鈣、磷、鐵，是一個很棒的抗氧化來源。



## Store and load the index

In [ ]:
# Store
faiss.write_index(index, "practice_index.faiss")
# Load
index = faiss.read_index("practice_index.faiss")

# Integrate RAG and LLM

In [11]:
texts = [
    "藍靛果屬於水果類，富有豐富的維生素A、C、E及礦物質鉀、鈣、磷、鐵，是一個很棒的抗氧化來源。",
    "台灣芒果的盛產期為每年的5月至7月，主要產區集中在台南、屏東和高雄。",
    "影集《小學風雲》自2021年首播以來，便以幽默風趣卻不失真誠的敘事風格，在美劇中脫穎而出，成為第80屆金球獎大贏家。",
    "Hulu登陸台灣Disney+，正式取代原Star品牌，目的是擴展Hulu的全球影響力。",
]

In [12]:
response = client.embeddings.create(model=EMBEDDING_MODEL, input=texts)
embeddings = np.array([d.embedding for d in response.data]).astype("float32")

# normalize → cosine similarity
faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

## User ask -> search

In [13]:
query = "藍靛果富含什麼東西？"

query_vec = client.embeddings.create(
    model=EMBEDDING_MODEL,
    input=query
).data[0].embedding

query_vec = np.array([query_vec]).astype("float32")
faiss.normalize_L2(query_vec)

top_k = 2
scores, indices = index.search(query_vec, top_k)
retrieved_texts = [texts[i] for i in indices[0]]

print("找到相關內容：")
for t in retrieved_texts:
    print("-", t)

找到相關內容：
- 藍靛果屬於水果類，富有豐富的維生素A、C、E及礦物質鉀、鈣、磷、鐵，是一個很棒的抗氧化來源。
- 台灣芒果的盛產期為每年的5月至7月，主要產區集中在台南、屏東和高雄。


## Send retrieval result to GPT

In [14]:
context = "\n".join(retrieved_texts)

prompt = f"""你是一位知識助理，請根據以下資料回答問題。
若無法從資料中確定答案，請明確說「資料中沒有提到」。

資料：
{context}

問題：{query}
"""

response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[
        {"role": "system", "content": "你是一位中文知識助理，回答要簡潔精確。"},
        {"role": "user", "content": prompt}
    ],
    max_completion_tokens=MAX_COMPLETION_TOKENS,
    temperature=0.2
)

print("\n最終回答：", response.choices[0].message.content)


最終回答： 藍靛果富含豐富的維生素A、C、E及礦物質鉀、鈣、磷、鐵。
